# Create Dataset from S3

Install pre-req software


In [ ]:
!pip install sagemaker --upgrade
!pip install "transformers>=4.4.2" 
!pip install s3fs --upgrade
!pip install datasets --upgrade 
#!pip install "librosa==0.9.1librosa"
!pip install torch # framework is required for transformer 
!pip install torchaudio

!conda install -y -c conda-forge librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 33.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.24 requires botocore==1.29.24, but you have botocore 1.29.28 which is incompatible.
awscli 1.27.24 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.24 requires rsa<4.8,>=3.1.2, but you have rsa 4.9 which is incompatible.
aiobotocore 2.4.1 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.28 which is incompatible.
  Using cached botocore-1.27.59-py3-none-any.whl (9.1 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.28
    Uninstalling botocore-1.29.28:
      Successf

In [ ]:
!conda install -c conda-forge libsndfile -y
!pip install soundfile

Set up S3 connection

In [ ]:
import json
import time
import boto3
import numpy as np
import random
import soundfile 
import sagemaker
import sagemaker.huggingface

BUCKET="radio-dataset" # please use your bucket name
PREFIX = "bbc" 
ROLE = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=BUCKET)

print(f"sagemaker role arn: {ROLE}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

Log in to huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torchaudio

from datasets import load_dataset, load_metric, Audio, DatasetDict,load_from_disk

from datasets.filesystems import S3FileSystem
s3 = S3FileSystem()


In [ ]:
datasetPath="s3://radio-dataset/bbc/clips/"
dataset = load_dataset("audiofolder", data_dir=datasetPath)

Test that dataset loaded